# Imports

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Bidirectional, LSTM, GRU, TimeDistributed, Activation, Flatten, Embedding
from keras.optimizers import Adam



/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Functions

In [2]:
# Prepare Glove File
def readGloveFile(gloveFile):
    with open(gloveFile, 'r') as f:
        wordToGlove = {}  # map from a token (word) to a Glove embedding vector
        wordToIndex = {}  # map from a token to an index
        indexToWord = {}  # map from an index to a token 

        for line in f:
            record = line.strip().split()
            token = record[0] # take the token (word) from the text line
            wordToGlove[token] = np.array(record[1:], dtype=np.float64) # associate the Glove embedding vector to a that token (word)

        tokens = sorted(wordToGlove.keys())
        for idx, tok in enumerate(tokens):
            kerasIdx = idx + 1  # 0 is reserved for masking in Keras (see above)
            wordToIndex[tok] = kerasIdx # associate an index to a token (word)
            indexToWord[kerasIdx] = tok # associate a word to a token (word). Note: inverse of dictionary above

    return wordToIndex, indexToWord, wordToGlove

In [3]:
# Create Pretrained Keras Embedding Weights Matrix
def createPretrainedEmbeddingMatrix(wordToGlove, wordToIndex):
    vocabLen = len(wordToIndex) + 1  # adding 1 to account for masking
    embDim = next(iter(wordToGlove.values())).shape[0]  # works with any glove dimensions (e.g. 50)

    embeddingMatrix = np.zeros((vocabLen, embDim))  # initialize with zeros
    for word, index in wordToIndex.items():
        embeddingMatrix[index, :] = wordToGlove[word] # create embedding: word index to Glove word embedding

    return vocabLen, embDim, embeddingMatrix

In [4]:
# Embedding
def getEncodedDocs(docs):
    encoded_docs = []

    tokenizer = RegexpTokenizer(r'\w+')
    for doc in docs:
        encoded_doc = []
        for word in tokenizer.tokenize(doc.lower()):
            index = wordToIndex[word]
            if index is not None:
                encoded_doc.append(index)
            else:
                encoded_doc.append(0)
        encoded_docs.append(encoded_doc)

    return encoded_docs

# Import GloVe Pretrained dataset and create Embedding Weight Matrix

In [5]:
max_length = 20
wordToIndex, indexToWord, wordToGlove = readGloveFile("glove/glove.6B.100d.txt")
vocabLen, embDim, embeddingMatrix = createPretrainedEmbeddingMatrix(wordToGlove, wordToIndex)

# Import intents file

In [6]:
import json
with open('PharmacyDataset.json') as json_data:
    intents = json.load(json_data)

# Padding, Encoding and Preparing final X and Y data for Training

In [7]:
classes = []
encodedUtterances = []

# loop through each sentence in our intents utterances
for intent in intents['intents']:
    classes.append(intent['intent'])
    encoded_docs = getEncodedDocs(intent['utterances'])
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    encodedUtterances.append(padded_docs)

# Exporting Classes / Intents array

In [8]:
import json
with open('Classes.json', 'w') as fp:
    json.dump(classes, fp)

In [9]:
currentClass = 0
train_x = []
train_y = []

for intent in classes:
    y = [0] * len(classes)
    y[currentClass] = 1

    for vector in encodedUtterances[currentClass]:
        train_x.append(vector)
        train_y.append(y)

    currentClass += 1

vectorSize = len(train_x[0])

train_X = np.array(train_x)
train_Y = np.array(train_y)

print(classes, "classes")
print(vectorSize, "vector size")
print(len(train_x), len(train_x[0]), "x")
print(len(train_y), len(train_y[0]), "y")
print(train_X.shape)
print(train_Y.shape)

([u'greeting', u'goodbye', u'thanks', u'options', u'adverse_drug', u'blood_pressure', u'blood_pressure_search', u'pharmacy_search'], 'classes')
(20, 'vector size')
(26, 20, 'x')
(26, 8, 'y')
(26, 20)
(26, 8)


# Prepare and Compile Keras / TensorFlow model

In [10]:
#sequence_input = Input(shape=(None, len(train_x[0])), dtype='float')
#bidiGru = Bidirectional(GRU(100))(sequence_input)
#preds = Dense(len(train_y[0]), activation='softmax')(bidiGru)
#model = Model(sequence_input, preds)
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])


# NB using LSTM as CoreML only support LSTM in Bidirectional layer


model = Sequential([
                    Embedding(vocabLen, embDim, weights=[embeddingMatrix], input_length=max_length, trainable=False),
                    GRU(embDim, batch_size=1, input_shape=(None, embDim), return_sequences=True),
                    #LSTM(embDim, batch_size=1, input_shape=(None, embDim), return_sequences=True),
                    TimeDistributed(Dense(64)),
                    Activation('relu'),
                    TimeDistributed(Dense(32)),
                    Activation('relu'),
                    Flatten(),
                    Dense(len(train_y[0]), activation='softmax')
                   ])

print("model fitting - Monodirectional GRU")
model.summary()


model fitting - Monodirectional GRU
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           40000100  
_________________________________________________________________
gru_1 (GRU)                  (None, 20, 100)           60300     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 64)            6464      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 64)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 32)            2080      
_________________________________________________________________
activation_2 (Activation)    (None, 20, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)

In [11]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model

In [12]:
model.fit(train_X, train_Y, epochs=50)

Epoch 1/50
26/26 [==============================] - 0s 18ms/step - loss: 2.0878 - acc: 0.0769
Epoch 2/50
26/26 [==============================] - 0s 587us/step - loss: 2.0433 - acc: 0.1154
Epoch 3/50
26/26 [==============================] - 0s 699us/step - loss: 2.0057 - acc: 0.2308
Epoch 4/50
26/26 [==============================] - 0s 766us/step - loss: 1.9694 - acc: 0.3846
Epoch 5/50
26/26 [==============================] - 0s 632us/step - loss: 1.9349 - acc: 0.4231
Epoch 6/50
26/26 [==============================] - 0s 694us/step - loss: 1.8973 - acc: 0.5000
Epoch 7/50
26/26 [==============================] - 0s 794us/step - loss: 1.8587 - acc: 0.5385
Epoch 8/50
26/26 [==============================] - 0s 802us/step - loss: 1.8172 - acc: 0.5000
Epoch 9/50
26/26 [==============================] - 0s 696us/step - loss: 1.7733 - acc: 0.5385
Epoch 10/50
26/26 [==============================] - 0s 819us/step - loss: 1.7262 - acc: 0.6154
Epoch 11/50
26/26 [==============================]

# Save Model

In [13]:
model.save('SwiftNLCGloveRNN.h5')

# Test Model

In [14]:
# evaluate the model
loss, accuracy = model.evaluate(train_X, train_Y, verbose=0)
print('Loss: %f ' % (loss*100))
print('Accuracy: %f ' % (accuracy*100))

Loss: 2.206458 
Accuracy: 100.000000 


In [15]:
# Test Predict Function
def predict(sentence):
    test_docs = getEncodedDocs([sentence])
    padded_test_docs = pad_sequences(test_docs, maxlen=max_length, padding='post')
    #print(padded_test_docs)

    y_pred = model.predict(padded_test_docs)
    #print(y_pred)

    max_value = max(y_pred[0])
    #print(max_value)

    max_index = y_pred[0].tolist().index(max_value)
    #print(max_index)

    #print(intents['intents'][max_index]['intent'])
    return intents['intents'][max_index]['intent'], max_value


In [16]:
# Cheating TEST
for intentGroup in intents["intents"]:
    print(intentGroup["intent"])
    for phrase in intentGroup["utterances"]:
        intent, prob = predict(phrase)
        print(phrase, intent, prob)


greeting
(u'Hi', u'greeting', 0.9625235)
(u'How are you', u'greeting', 0.99039733)
(u'Is anyone there?', u'greeting', 0.9959395)
(u'Hello', u'greeting', 0.87115234)
(u'Good day', u'greeting', 0.961964)
goodbye
(u'Bye', u'goodbye', 0.95516914)
(u'See you later', u'goodbye', 0.9564257)
(u'Goodbye', u'goodbye', 0.895754)
thanks
(u'Thanks', u'thanks', 0.9140722)
(u'Thank you', u'thanks', 0.97892797)
(u"That's helpful", u'thanks', 0.99135464)
options
(u'How you could help me?', u'options', 0.9980263)
(u'What you can do?', u'options', 0.99963856)
(u'What help you provide?', u'options', 0.9922111)
adverse_drug
(u'How to check Adverse drug reaction?', u'adverse_drug', 0.9974745)
(u'List all drugs suitable for patient with adverse reaction', u'adverse_drug', 0.99812835)
(u'Which drugs dont have adverse reaction?', u'adverse_drug', 0.9992101)
blood_pressure
(u'Open blood pressure module', u'blood_pressure', 0.9995524)
(u'I want to log blood pressure results', u'blood_pressure', 0.99935204)
(u'Bl

In [17]:
#Test Dataset TEST

testDataset = [ "Display blood values for patient",
                "That was very helpful",
                "Show me the patient blood pressure results",
                "Search for a pharmacy please",
                "How can you help me" ]



for phrase in testDataset:
    intent, prob = predict(phrase)
    print(phrase, intent, prob)
    

('Display blood values for patient', u'blood_pressure_search', 0.93922055)
('That was very helpful', u'greeting', 0.9139301)
('Show me the patient blood pressure results', u'blood_pressure', 0.9622778)
('Search for a pharmacy please', u'pharmacy_search', 0.9978824)
('How can you help me', u'options', 0.99959654)


# Export Word Embedding Array

In [18]:
for i in range(0,10):
    print(wordToIndex.keys()[i], wordToIndex[wordToIndex.keys()[i]], i)
    
import json
with open('Words.json', 'w') as fp:
    json.dump(wordToIndex, fp)


('biennials', 75684, 0)
('tripolitan', 365444, 1)
('biysk', 77319, 2)
('woode', 389559, 3)
('verplank', 377800, 4)
('mdbo', 239051, 5)
('sowell', 338477, 6)
('mdbu', 239054, 7)
('soestdijk', 336526, 8)
('spiders', 339422, 9)


# Export Model using CoreML Tools

In [19]:
import coremltools
coreml_model = coremltools.converters.keras.convert(model, input_names="vectors", output_names="entities")
coreml_model

0 : embedding_1_input, <keras.engine.topology.InputLayer object at 0x146a2d550>
1 : embedding_1, <keras.layers.embeddings.Embedding object at 0x1107f9c50>
2 : gru_1, <keras.layers.recurrent.GRU object at 0x127477090>
3 : time_distributed_1, <keras.layers.wrappers.TimeDistributed object at 0x146a2d150>
4 : activation_1, <keras.layers.core.Activation object at 0x146a2d190>
5 : time_distributed_2, <keras.layers.wrappers.TimeDistributed object at 0x146a2d2d0>
6 : activation_2, <keras.layers.core.Activation object at 0x146a2d310>
7 : flatten_1, <keras.layers.core.Flatten object at 0x146a2d350>
8 : dense_3, <keras.layers.core.Dense object at 0x146a2d3d0>
9 : dense_3__activation__, <keras.layers.core.Activation object at 0x1291f7b10>


input {
  name: "vectors"
  type {
    multiArrayType {
      shape: 1
      dataType: DOUBLE
    }
  }
}
input {
  name: "gru_1_h_in"
  type {
    multiArrayType {
      shape: 100
      dataType: DOUBLE
    }
    isOptional: true
  }
}
output {
  name: "entities"
  type {
    multiArrayType {
      shape: 8
      dataType: DOUBLE
    }
  }
}
output {
  name: "gru_1_h_out"
  type {
    multiArrayType {
      shape: 100
      dataType: DOUBLE
    }
  }
}

In [20]:
coreml_model.save('SwiftNLCGloveRNN.mlmodel')